In [1]:
import uuid
import os
from IPython.display import display, clear_output, Markdown
from time import sleep
import json

In [24]:
mds = os.listdir("./wikis")

In [2]:
def read_file(md):
    with open(f"wikis/{md}", "r", encoding="utf-8") as file:
        doc = file.read()
    return doc

In [26]:
def parse_document(doc):
    lines = doc.strip().split('\n')
    docs = []
    source = lines[0].strip()
    if lines:
        # The first line is the source
        current_content = []
        current_headline = ''

        for line in lines[1:]:
            stripped_line = line.strip()
            if not stripped_line:
                continue
            if stripped_line.startswith("#"):
                if current_content:

                    docs.append(
                        {
                        "source": source,
                        "content": "\n".join(current_content),
                        "headline": current_headline,
                        "length": len("\n".join(current_content)),
                        "id": str(uuid.uuid4())

                        }
                    )

                    current_content = []
                current_headline = stripped_line.lstrip("#").strip()
            else:
                current_content.append(stripped_line)

        if current_content:


            docs.append(
                {
                "source": source,
                "content": "\n".join(current_content),
                "headline": current_headline,
                "length": len("\n".join(current_content)),
                "id": str(uuid.uuid4())
                }
            )

    return docs

In [27]:
md = read_file(mds[1])
test = parse_document(md)

In [28]:
documents = []

for md in mds:
    doc = read_file(md)
    docs = parse_document(doc)
    for doc in docs:
        documents.append(
            {
                "source": doc["source"],
                "content": doc["content"],
                "headline": doc["headline"],
                "id": doc["id"],
                "length": doc["length"],
            }
        )

In [29]:
documents[1]["content"]

'The Autobahn is a [network of interstate highways in Germany](https://en.m.wikipedia.org/wiki/Autobahn#/media/File%3AAutobahnen_in_Deutschland.svg) with a total length of more than 8,000 miles. [65%](https://en.wikipedia.org/wiki/Autobahn#Speed_limits) of the Autobahn has no speed limit. How safe can that be?\nVehicles traveled 147 billion miles on the Autobahn in 2015. 322 people died = 2.19 deaths per billion miles.\nIn the US, vehicles travelled 757 billion miles on interstate highways. 3,837 people died = 5.07 deaths per billion miles.\nThat means: If you drive on the interstate, your likelihood to die is 131% higher than for the same distance on the Autobahn.\n*sources:*\nStatistisches Bundesamt: [Unfallentwicklung auf deutschen Straßen 2015](https://www.destatis.de/DE/PresseService/Presse/Pressekonferenzen/2016/Unfallentwicklung_2015/Pressebroschuere_unfallentwicklung.pdf?__blob=publicationFile)\nNational Highway Traffic Safety Administration: [Fatal Crashes by STATE and Road Fu

In [32]:
for doc in documents:
    if doc["content"] is None:
        print(doc["id"])
    elif doc["headline"] is None:
        print(doc["id"])
    elif doc["length"] is None:
        print(doc["id"])
    elif doc["source"] is None:
        print(doc["id"])
    else:
        pass


In [31]:
for doc in documents:
    display(Markdown(f"## {doc['headline']}"))
    display(Markdown(f"{doc['content']}"))
    display(Markdown(f"{doc['id']}"))
    sleep(1)
    clear_output(wait=True)

## Cards

Your bank account will come with at least one bank card, and you can usually have more. The following card types are common in Germany:
* Debit cards. Any amount paid with this card is immediately debited from the associated bank account.
* Charge cards. Any amount paid with this card is automatically debited from the associated bank account at the end of the month.
* Prepaid credit cards. The owner transfers money onto the credit card's account, which is then debited when the card is used to pay for something.
* Classical credit cards (also called "rolling credit cards"). Any amount paid with this card is tracked as debt, and any debt which isn't repaid by the end of the month (most cards require a minimum monthly payment) accrues interest.
The cards you will typically have are:
* A [Girocard](https://en.wikipedia.org/wiki/Girocard) debit card (also called an "EC card"), which is your main card for cashless payments. This card is almost always free. Note that N26 is the only major bank which doesn't issue a Girocard - instead, they give you a Maestro debit card, which is somewhat useful, but not as universally accepted as a Girocard.
* A credit card, which you would commonly use for online shopping and payments outside the Eurozone. This card often (but not always) has an annual fee. Most banks will offer you a charge card; classical credit cards exist, but are comparatively rare. Both of these cards are generally only given to people with a good Schufa score (credit rating); for other people, prepaid credit cards are an option. If you don't necessarily want to use your bank's credit card, you can use a comparison shopping site like [Cardscout](https://www.cardscout.de/) to compare offers - however, for credit cards without a monthly fee, make sure you understand the fine print to avoid paying unexpected fees.
Many cards will also have options for limited (limited to 25 €) contactless payment, and many banks let you connect your cards to smartphone payment systems like Apple Pay or Google Pay.
Credit cards with cashback or other benefits are fairly uncommon in Germany.

14aad13d-3ff3-4efe-a6b3-1a614052ce5b

KeyboardInterrupt: 

In [34]:
with open('documents.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [9]:
def parse_document(doc):
    lines = doc.strip().split('\n')
    docs = []
    if lines:
        current_headline = ''
        current_question = ''
        current_answer = ''
        inside_question = False
        inside_answer = False

        for line in lines[1:]:
            stripped_line = line.strip()
            if not stripped_line:
                # Skip empty lines
                continue

            if stripped_line.startswith("**Q"):
                # If we were inside an answer, save it before starting a new question
                if inside_answer:
                    docs.append({
                        "headline": current_headline,
                        "question": current_question.lstrip(": ").strip(),
                        "content": current_answer.lstrip(":** ").strip(),
                        "length": len("\n".join(current_answer)),
                        "source": "https://www.reddit.com/r/germany/wiki/faq",
                        "id": str(uuid.uuid4())
                    })
                    current_answer = ''
                # Start collecting the question
                current_question = stripped_line[3:].rstrip("**").strip()  # Remove **Q and trailing **
                inside_question = True
                inside_answer = False

            elif stripped_line.startswith("**A"):
                # Finish the question and start collecting the answer
                inside_question = False
                inside_answer = True
                current_answer = stripped_line[3:].rstrip("**").strip()  # Remove **Q and trailing **

            elif stripped_line.startswith("#"):
                # If we were inside an answer, save it before starting a new headline
                if inside_answer or current_question:
                    docs.append({
                        "headline": current_headline,
                        "question": current_question.lstrip(": ").strip(),
                        "content": current_answer.lstrip(":** ").strip(),
                        "length": len("\n".join(current_answer)),
                        "source": "https://www.reddit.com/r/germany/wiki/faq",
                        "id": str(uuid.uuid4())
                    })
                    current_question = ''
                    current_answer = ''
                    inside_question = False
                    inside_answer = False

                # New headline starts
                current_headline = stripped_line.lstrip("#").strip().rstrip("**").strip()  # Remove **Q and trailing **
                
            else:
                # Accumulate content based on current state
                if inside_question:
                    current_question += "\n" + stripped_line
                elif inside_answer:
                    current_answer += "\n" + stripped_line

        # Add any remaining content at the end of the document
        if inside_answer or current_question:

            docs.append({
                "headline": current_headline,
                "question": current_question.lstrip(": ").strip(),
                "content": current_answer.lstrip(":** ").strip(),
                "length": len("\n".join(current_answer)),
                "source": "https://www.reddit.com/r/germany/wiki/faq",
                "id": str(uuid.uuid4())

            })

    return docs

In [10]:
docs = read_file("faq.md")

In [11]:
doc = parse_document(docs)

In [12]:
print(doc)

[{'headline': 'Travelling', 'question': 'What kind of clothing should I bring?', 'content': 'Germany has temperatures ranging from ~-15°C to ~34°C (5°F-93°F). Both of these are extremes that occur only during a few weeks in the year. Usually having a warm jacket during the winter, and a wind jacket or raincoat during the rest of the year should be fine. In the summer and spring, bring both long and short clothing, in autumn and winter, bring long clothing. What you should not bring are baseball caps. They look just weird and nobody really wears them. There is snow in the winter, and a lot of rain through the year.', 'length': 1063, 'source': 'https://www.reddit.com/r/germany/wiki/faq', 'id': '668dc206-4248-4554-bddb-238ee9eaa0df'}, {'headline': 'Travelling', 'question': 'Is it possible to hitchhike through Germany?', 'content': 'While hitchhiking isn\'t that common any more, it should still be possible to do it. Hitchhike from Autobahnrastplatz (motorway station) to Autobahnrastplatz, 

In [14]:
doc[0]

{'headline': 'Travelling',
 'question': 'What kind of clothing should I bring?',
 'content': 'Germany has temperatures ranging from ~-15°C to ~34°C (5°F-93°F). Both of these are extremes that occur only during a few weeks in the year. Usually having a warm jacket during the winter, and a wind jacket or raincoat during the rest of the year should be fine. In the summer and spring, bring both long and short clothing, in autumn and winter, bring long clothing. What you should not bring are baseball caps. They look just weird and nobody really wears them. There is snow in the winter, and a lot of rain through the year.',
 'length': 1063,
 'source': 'https://www.reddit.com/r/germany/wiki/faq',
 'id': '668dc206-4248-4554-bddb-238ee9eaa0df'}

In [18]:
for d in doc:
    print(d["headline"])
    print(d["content"])
    print(d["question"])
    print(d["source"])
    sleep(1)
    clear_output(wait=True)

Genealogy
Yes, victims of National Socialist denationalization measures and their descendants have the right to be renationalized in line with Article 116 (2) of the Basic Law even if this means multiple nationality. There is no need to prove knowledge of the German language. Nevertheless it is examined whether the German nationality of the ancestor could have been lost for reasons unrelated to National Socialism. Were this the case, the descendants would have no right to German citizenship. [source](http://www.auswaertiges-amt.de/sid_8102715B432128CA20217EC07AF2F4C5/EN/Infoservice/FAQ/Staatsangehoerigkeit/16-Ausbuergerung.html?nn=479790)
I/my ancestors was/were denationalized by the National Socialists. Can I get my German citizenship back?
https://www.reddit.com/r/germany/wiki/faq


In [19]:
for d in  doc:
    if d["content"] is None:
        print(doc["id"])
    elif d["headline"] is None:
        print(doc["id"])
    elif d["length"] is None:
        print(doc["id"])
    elif d["question"] is None:
        print(doc["id"])
    else:
        pass

In [20]:
with open('faq.json', 'wt') as f_out:
    json.dump(doc, f_out, indent=2)

In [21]:
with open("doc_with_q_4o-mini.json", 'rt') as f_in:
    list1 = json.load(f_in)

In [22]:
with open("faq.json", 'rt') as f_in:
    list2 = json.load(f_in)

In [23]:
with open("doc_with_q_lama.json", 'rt') as f_in:
    list3 = json.load(f_in)

In [24]:
full_gpt = list1 + list2

In [25]:
len(full_gpt)

661

In [26]:
full_lama = list3 + list2

In [27]:
len(full_lama)

661

In [28]:
with open('docs_with_q_4o-mini.json', 'wt') as f_out:
    json.dump(full_gpt, f_out, indent=2)

In [30]:
with open("docs_with_q_lama.json", 'wt') as f_out:
    json.dump(full_lama, f_out, indent=2)

In [31]:
for d in  full_lama:
    if d["content"] is None:
        print(doc["id"])
    elif d["headline"] is None:
        print(doc["id"])
    elif d["length"] is None:
        print(doc["id"])
    elif d["question"] is None:
        print(doc["id"])
    else:
        pass

In [32]:
for d in  full_gpt:
    if d["content"] is None:
        print(doc["id"])
    elif d["headline"] is None:
        print(doc["id"])
    elif d["length"] is None:
        print(doc["id"])
    elif d["question"] is None:
        print(doc["id"])
    else:
        pass